# Predicting House Prices Using the Ames Iowa Dataset

## Exploratory data analysis and initial cleaning

In [1]:
# Importing and specifying some of the relevant elements

seed = 0
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_csv('../input/housing/AmesHousing.csv')

**Note**: To understand the features of the dataset refer to its [data-description file](https://s3.amazonaws.com/dq-content/307/data_description.txt).

In [3]:
print("Rows and columns:", df.shape)
print("Total null values:", df.isnull().sum().sum())

Rows and columns: (2930, 82)
Total null values: 13997


In [4]:
# Determining the counts, percentages, and data types of null values

mask = df.isnull()
total = mask.sum()
percent = mask.mean() * 100
dtypes = df.dtypes

missing_data = pd.concat([total, percent, dtypes], axis=1, join='outer',\
                         keys=['count_null', 'percent_null', 'data_type'])
missing_data.sort_values(by='percent_null', ascending=False, inplace=True)
missing_data.T

,Pool QC,Misc Feature,Alley,Fence,Fireplace Qu,Lot Frontage,Garage Cond,Garage Finish,Garage Yr Blt,Garage Qual,Garage Type,Bsmt Exposure,BsmtFin Type 2,Bsmt Qual,Bsmt Cond,BsmtFin Type 1,Mas Vnr Area,Mas Vnr Type,Bsmt Full Bath,Bsmt Half Bath,BsmtFin SF 1,Garage Cars,Electrical,Total Bsmt SF,Bsmt Unf SF,BsmtFin SF 2,Garage Area,Paved Drive,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Sale Condition,Sale Type,Yr Sold,Mo Sold,Misc Val,Functional,Fireplaces,Pool Area,Screen Porch,3Ssn Porch,Enclosed Porch,Open Porch SF,Wood Deck SF,Order,Heating QC,Gr Liv Area,Overall Qual,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Cond,Low Qual Fin SF,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Exter Qual,Exter Cond,Foundation,Heating,PID,Central Air,1st Flr SF,2nd Flr SF,SalePrice
count_null,2917,2824,2732,2358,1422,490,159,159,159,159,157,83,81,80,80,80,23,23,2,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
percent_null,99.5563,96.3823,93.2423,80.4778,48.5324,16.7235,5.42662,5.42662,5.42662,5.42662,5.35836,2.83276,2.76451,2.73038,2.73038,2.73038,0.784983,0.784983,0.0682594,0.0682594,0.0341297,0.0341297,0.0341297,0.0341297,0.0341297,0.0341297,0.0341297,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
data_type,object,object,object,object,object,float64,object,object,float64,object,object,object,object,object,object,object,float64,object,float64,float64,float64,float64,object,float64,float64,float64,float64,object,int64,int64,int64,int64,object,int64,object,object,int64,int64,int64,object,int64,int64,int64,int64,int64,int64,int64,int64,object,int64,int64,int64,object,int64,object,object,object,object,object,object,object,object,object,object,object,int64,int64,int64,int64,object,object,object,object,object,object,object,object,int64,object,int64,int64,int64


In [5]:
# Making the column names lower case

df.columns = df.columns.str.lower().\
                     str.replace(' ', '_')

In [6]:
# Isolating the categorical columns


categorical_cols = list(df.select_dtypes\
                                (include=['object', 'category']).columns)

In [7]:
# Making the categorical values lower case

for col in categorical_cols:
    df[col] = df[col].str.lower()

In [8]:
df.head(3)

,order,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice
0,1,526301100,20,rl,141.0,31770,pave,NaN,ir1,lvl,allpub,corner,gtl,names,norm,norm,1fam,1story,6,5,1960,1960,hip,compshg,brkface,plywood,stone,112.0,ta,ta,cblock,ta,gd,gd,blq,639.0,unf,0.0,441.0,1080.0,gasa,fa,y,sbrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,ta,7,typ,2,gd,attchd,1960.0,fin,2.0,528.0,ta,ta,p,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,wd,normal,215000
1,2,526350040,20,rh,80.0,11622,pave,NaN,reg,lvl,allpub,inside,gtl,names,feedr,norm,1fam,1story,5,6,1961,1961,gable,compshg,vinylsd,vinylsd,none,0.0,ta,ta,cblock,ta,ta,no,rec,468.0,lwq,144.0,270.0,882.0,gasa,ta,y,sbrkr,896,0,0,896,0.0,0.0,1,0,2,1,ta,5,typ,0,NaN,attchd,1961.0,unf,1.0,730.0,ta,ta,y,140,0,0,0,120,0,NaN,mnprv,NaN,0,6,2010,wd,normal,105000
2,3,526351010,20,rl,81.0,14267,pave,NaN,ir1,lvl,allpub,corner,gtl,names,norm,norm,1fam,1story,6,6,1958,1958,hip,compshg,wd sdng,wd sdng,brkface,108.0,ta,ta,cblock,ta,ta,no,alq,923.0,unf,0.0,406.0,1329.0,gasa,ta,y,sbrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,gd,6,typ,0,NaN,attchd,1958.0,unf,1.0,312.0,ta,ta,y,393,36,0,0,0,0,NaN,NaN,gar2,12500,6,2010,wd,normal,172000


In [9]:
# Removing parenthetical elements from the values in the ms_zoning column

df.ms_zoning = df.ms_zoning.map(lambda x: x[0] if '(' in x else x)

## Feature engineering

During this step, we will perform the following tasks:

1. Delete every column where at least 5% values are missing.

2. Delete every text column where 1 or more values are missing.

3. Fill every numerical column with missing values with the most common value in that column.

4. Add new features.

5. Create dummy variables.

In [10]:
# Deleting every column where at least 5% values are missing

num_missing = df.isnull().sum()

drop_missing_cols = num_missing[(num_missing >\
                                   len(df)/20)].sort_values()

df = df.drop(drop_missing_cols.index, axis=1)

In [11]:
numerical_cols = list(df.select_dtypes\
                                (include=['int64', 'float64', 'int32']).columns)

categorical_cols = list(df.select_dtypes\
                                (include=['object', 'category']).columns)

In [12]:
# Delete every text column where 1 or more values are missing

text_null_counts = df[categorical_cols].isnull().sum().\
                   sort_values(ascending=False)

drop_missing_cols_text = text_null_counts[text_null_counts > 0]

df = df.drop(drop_missing_cols_text.index, axis=1)

In [13]:
# Filling every numerical column that contains missing values 
# with the most common value in that column

num_null_counts = df[numerical_cols].isnull().sum().\
                   sort_values(ascending=False)

num_cols_to_fill = num_null_counts[num_null_counts > 0]

replacement_values_dict = df[num_cols_to_fill.index].mode().\
                          to_dict(orient='records')[0]

df = df.fillna(replacement_values_dict)

In [14]:
df.isnull().sum().sum()

0

In [15]:
# Adding new features

df['years_before_sale'] = df.yr_sold - df.year_built
df['years_since_remod'] = df.yr_sold - df['year_remod/add']

In [16]:
print('Index of each row with a negative years_before_sale:')
print(df.years_before_sale[df.years_before_sale < 0])

print('\nIndex of each row with negative years_since_remod:')
print(df.years_since_remod[df.years_since_remod < 0])

Index of each row with a negative years_before_sale:
2180   -1
Name: years_before_sale, dtype: int64

Index of each row with negative years_since_remod:
1702   -1
2180   -2
2181   -1
Name: years_since_remod, dtype: int64


In [17]:
# Deleting rows with negative values

df.drop([1702, 2180, 2181], axis=0, inplace=True)

# Deleting irrelevant columns

cols_to_delete = ['order', 'pid', 'yr_sold', 'year_built', 'mo_sold', \
                  'year_remod/add', 'sale_condition', 'sale_type']
df.drop(cols_to_delete, axis=1, inplace=True)

In [18]:
# Determining the correlation between saleprice and other numerical columns

# Isolating numerical columns

numerical_cols = list(df.select_dtypes\
                                (include=['int64', 'float64', 'int32']).columns)

In [19]:
# Calculating correlation

abs_corr_coeffs = df[numerical_cols].corr()['saleprice'].abs().sort_values()

# Removing columns whose absolute correlation coefficients are less than 0.4

df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1, inplace=True)

In [20]:
# Identifying columns in the updated dataframe that can generate 
# a manageable number of dummy variables

# Identifying categorical columns

cols_for_dummies= []

sample_cols = ['ms_zoning', 'street', 'alley', 'land_contour', 'lot_config',\
 'neighborhood', 'condition_1', 'condition_2', 'bldg_type',\
 'house_style', 'roof_style', 'roof_matl', 'exterior_1st',\
 'exterior_2nd', 'foundation', 'heating', 'central_air', 'garage_type']

for col in sample_cols:
    if col in df.columns:
        cols_for_dummies.append(col)
        
# Determining the number of unique values in each categorical column

uniqueness_counts = df[cols_for_dummies].apply\
                    (lambda col: len(col.value_counts())).sort_values()
    
# Removing the categorical columns that have > 10 unique values
cols_to_remove = uniqueness_counts[uniqueness_counts > 10].index
    
df.drop(cols_to_remove, axis=1, inplace=True)

In [21]:
# Changing the datatype of object type columns to category

obj_cols = df.select_dtypes(include=['object'])

for col in obj_cols:
    df[col] = df[col].astype('category')

In [22]:
# Creating dummies

cols_for_dummies = df.select_dtypes(include=['category'])

for col in cols_for_dummies:
    df = pd.get_dummies(df, columns = [col], prefix=col)

## Training models

In [23]:
# Simple linear regressor

from sklearn.linear_model import LinearRegression

numeric_df = df.select_dtypes(include=['integer', 'float'])
features = numeric_df.columns.drop("saleprice")

lr = LinearRegression()

In [24]:
train = df[:1460]
test = df[1460:]

lr.fit(train[features], train.saleprice);

In [25]:
from sklearn.metrics import mean_squared_error

predictions = lr.predict(test[features])
mse = mean_squared_error(test.saleprice, predictions)
rmse = np.sqrt(mse)

In [26]:
rmse

33367.28718340385

In [27]:
# Linear regression with data split according to the k-fold strategy

from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True)
rmse_values = []
for train_index, test_index, in kf.split(df):
    train = df.iloc[train_index]
    test = df.iloc[test_index]
    lr.fit(train[features], train.saleprice)
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test.saleprice, predictions)
    rmse = np.sqrt(mse)
    rmse_values.append(rmse)
print(rmse_values)
avg_rmse = np.mean(rmse_values)
print(avg_rmse)

[26343.14753894019, 30025.88326701, 34299.73699146064, 26095.929457588143]
29191.17431374974
